# Exploración de indicadores que reflejen la incidencia de Covid-19 en Venezuela usando fuentes de datos no oficiales.

# Google Trends API - pytrends 

Twitter permite acceder a información pública mediante las API (Application Programming Interfaces), en español, interfaz de programación de aplicaciones, éstas dependen de las aplicaciones que se van a utilizar y de los permisos asignados. 

Este notebook de Jupyter contiene todo el código utilizado para extraer tweets por medio de la API de Twitter por medio de la biblioteca [Twarc](https://pypi.org/project/twarc/).

### Pasos a seguir para poder acceder a la API de Twiter. 
- Tener una cuenta en la red social de Twitter.
- Solicitar una cuenta de desarrollador. [Create a project app](https://developer.twitter.com/en/apps)
- Extraer las credenciales de permiso. 
[Academic Research Project](https://developer.twitter.com/en/docs/projects/overview#product-track)

___

**Advertencia: Los pasos en este notebook son repetitivos, ya que se documentaron todas las consultas para la extracción de los datos**.
___

## Configuración

Configurar el entorno importando las bibliotecas con las que se trabajaran. 

In [1]:
import twarc 
import pandas as pd
import datetime
import itertools
from twarc.client2 import Twarc2
import json

Hacer la autenticación por medio de las credenciales para poder hacer la solicitud de extracción.
___

**Nota: las credenciales utilizadas son de investigación académica. Al hacer la solicitud con otro tipo de credenciales, puede arrojar distintos resultados.** 

In [7]:
t = Twarc2(bearer_token="AAAAAAAAAAAAAAAAAAAAABmzOgEAAAAAAdXaEdw8lrDMBtPj%2BwY318SBBmk%3DCvdZpj9aQg2ORVIfm0glUMXBNSm1yoDl1Zp1ifSRh3Lf6j6QEm")
#Recomiendan no hacer las credenciales públicas

En principio, se define la consulta en el periodo comprendido entre el 13 de marzo de 2020 hasta el 10 de mayo de 2021, para ello se utilizará start_time y end_time, respectivamente. En algunas consultas estos parámetros cambian para obtener mejores resultados. 

In [4]:
start_time = datetime.datetime(2020, 3, 13, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)

#Estas variables deben estar en formato UTC

La consulta está determinada por: 

**Palabras claves**

- covid-19
- coronavirus
- infectado covid-19
- muerte covid-19
- bombona de oxígeno
- flujómetro de oxígeno
- saturación de oxígeno
- dexametasona
- GoFundMe venezuela covid-19
- recolectar dinero covid-19
- ayuda covid-19
- tratamiento covid-19
- UCI 
- unidad de cuidados intensivos
- servicio público covid-19 

Las consultas se especifican con y sin hastahtag(#), ya que con este operador se realiza una coincidencia exacta; las palabras claves se especifican con y sin tilde diacrítica, debido a que hay distinción entre las palabras que la posean y las que no, no obstante, no se hace distinción entre mayúsculas y minúsculas. Se filtran todos aquellos tweets que son retweets y. Adicionalmente, la búsqueda de todas las palabras claves, a excepción de las relacionadas con "GoFundMe", se hice local, es decir, se restringe la búsqueda al país Venezuela.

**Operadores y filtros**

- -is:retweet ---- Negación del operador is:retweet, usada para buscar solo tweets originales.
- OR ---- Conjunción (o), implica que los tweets coincidirán si cumple alguna de las condiciones. 
- place_country:VE ---- Usada para obtener tweets proveniente del país Venezuela.  
- has link ---- Con link adjunto. 

[Creación de consultas para los tweets de búsqueda](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#examples).
___

Se utiliza `t.search_all(query, star_time, end_time)` para hacer las consultas, donde el parámetro `query` está conformado por la palabra clave, filtros y operadores asociados. 

Por ejemplo: la siguiente consulta arroja tweets que contengan la palabra clave "covid-19", que provengan del "paí Venezuela", que sean tweets originales y del periodo de tiempo "13 de marzo de 2020 - 10 de mayo de 2021". 

~~~
search_results = t.search_all(query="covid-19 place_country:VE -is:retweet", start_time=start_time, end_time=end_time)
~~~

### Solicitud: covid-19, coronavirus

___ 

En principio, se a muestra los pasos detalladamente, luego se crea una función para aplicarlas a las distintas búsquedas.


In [ ]:
search_results = t.search_all(query="covid-19 place_country:VE -is:retweet OR coronavirus place_country:VE -is:retweet", start_time=start_time, end_time=end_time)

Obtener todas las páginas de resultados en un archivo formato .JSON. 

In [ ]:
for page in search_results:
    with open("../datos/datos_twitter/datos_twitter.json", "w+") as f:
        f.write(json.dumps(page) + "\n")

se convierte el archivo .json en un archivo .csv

In [ ]:
convertir a .cvs
df = pd.read_json(r'../datos/datos_twitter/datos_twitter.json')
df.to_csv (r'../datos/datos_twitter/datos_twitter.csv')     

A continuación, se crea una función que permita reproducir el procedimiento anterior.

In [3]:
def extraccion_datos_twitter(query, start_time, end_time, nombre):
    """Hacemos la solicitud"""
    search_results = t.search_all(query, start_time=start_time, end_time=end_time) 
    
    """Definimos la ruta y la variable que me da el nombre del archivo"""
    ruta = "../datos/datos_twitter/"
    nombre = ruta + nombre
    
    """Guardar resultados en .json"""
    for page in search_results:
        with open(nombre, "w+") as f:
            f.write(json.dumps(page) + "\n")  
    

### Solicitud: infectado covid-19, muerte covid-19

In [ ]:
query = "infectado covid-19 place_country:VE -is:retweet OR muerte covid-19 place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter2.json")

In [ ]:
query = "#infectadocovid-19 place_country:VE -is:retweet OR #muertecovid-19 place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter3.json")

### Solicitud: oxígeno, dexametasona

In [ ]:
query = "oxígeno place_country:VE -is:retweet OR dexametasona place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter4.json")

### Solicitud:bombona de oxígeno, dexametasona

In [ ]:
query="bombona de oxígeno place_country:VE -is:retweet OR dexametasona place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter5.json")

### Solicitud: GoFundMe

In [ ]:
query="GoFundMe venezuela covid-19 -is:retweet OR GoFundMe venezuela covid-19 has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter6.json")

In [ ]:
query="#GoFundMe venezuela covid-19 -is:retweet OR #GoFundMe venezuela covid-19 has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter7.json")

In [ ]:
start_time = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query="GoFundMe venezuela covid-19 -is:retweet OR GoFundMe venezuela covid-19 has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter8.json")

In [ ]:
query="#GoFundMe venezuela covid-19 -is:retweet OR #GoFundMe venezuela covid-19 has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter9.json")

### Solicitud: recolectar dinero covid-19, ayuda covid-19

In [ ]:
start_time = datetime.datetime(2020, 3, 13, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query = "recolectar dinero covid-19 place_country:VE -is:retweet OR ayuda covid-19 place_country:VE has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter10.json")

In [ ]:
start_time = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query = "#RecolectarDinero covid-19 place_country:VE -is:retweet OR #ayuda covid-19 place_country:VE has:links -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter11.json")

### Solicitud: flujómetro de oxígeno, saturación de oxígeno

In [ ]:
start_time = datetime.datetime(2020, 3, 13, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query="flujómetro de oxígeno place_country:VE -is:retweet OR saturación de oxígeno place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter12.json")

In [ ]:
query="flujometro de oxigeno place_country:VE -is:retweet OR saturacion de oxigeno place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter13.json")

In [ ]:
query="#flujmetroOxigeno place_country:VE -is:retweet OR saturacionOxigeno place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter14.json")

### Solicitud: servicio público

- servicio público 

In [ ]:
query="#ServicioPúblico covid-19 place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitte15.json")

In [ ]:
query="#ServicioPublico covid-19 place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter16.json")

In [ ]:
start_time = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter17.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter18.json")

In [ ]:
start_time = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 1, 31, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter19.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter20.json")

In [ ]:
start_time = datetime.datetime(2021, 2, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 2, 28, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter21.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter22.json")

In [ ]:
start_time = datetime.datetime(2021, 3, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 3, 31, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter23.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter24.json")

In [ ]:
start_time = datetime.datetime(2021, 4, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 4, 30, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter25.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter26.json")

In [ ]:
start_time = datetime.datetime(2021, 5, 1, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query="servicio público covid-19 place_country:VE -is:retweet OR servicio publico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter27.json")

In [ ]:
query="ServicioPublico covid-19 place_country:VE -is:retweet OR ServicioPúblico covid-19 place_country:VE -is:retweet" 
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter28.json")

In [ ]:
start_time = datetime.datetime(2020, 3, 13, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
search_results = t.search_all(query="servicio publico place_country:VE -is:retweet OR servicio publico place_country:VE -is:retweet", start_time=start_time, end_time=end_time)
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter29.json")

### Solicitud:  UCIS, unidad de cuidados intensivos

In [ ]:
start_time = datetime.datetime(2020, 3, 13, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
query="UCIS place_country:VE -is:retweet OR unidad de cuidados intensivos place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter30.json")

In [ ]:
query="#UCIS place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter31.json")

### Solicitud: tratamiento covid-19

In [ ]:
query="tratamiento covid-19 place_country:VE -is:retweet"
extraccion_datos_twitter(query, start_time, end_time, "datos_twitter32.json")

### Esto es una prueba 

In [8]:
start_time = datetime.datetime(2021, 4, 15, 0, 0, 0, 0, datetime.timezone.utc)
end_time = datetime.datetime(2021, 5, 10, 0, 0, 0, 0, datetime.timezone.utc)
search_results = t.search_all(query="#tratamientCovid place_country:VE -is:retweet OR #InfectadoCovid place_country:VE -is:retweet OR #MuerteCovid place_country:VE -is:retweet", start_time=start_time, end_time=end_time)

In [10]:
for page in search_results:
    with open("../datos/datos_twitter/prueba2.json", "w+") as f:
        f.write(json.dumps(page) + "\n")

In [20]:
#Qué estoy haciendo mal? 
df = pd.read_json(r'../datos/datos_twitter/prueba.json')
df.to_csv (r'../datos/datos_twitter/prueba1.csv')   

ValueError: arrays must all be same length